<a href="https://colab.research.google.com/github/aaaaxiaofei/notebooks/blob/main/gymCopy/gymCopy_qtable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gym Copy-v0 Environment

Conslusion: qTable is not a good approach

In [2]:
import gym
import numpy as np
import random

In [14]:
env = gym.make("Copy-v0")

In [69]:
# Get scalar action_size
action_size = 1
action_scale = list()
for i in range(len(env.action_space)):
  local_size = env.action_space[i].n
  action_size *= local_size

  action_scale = [x*local_size for x in action_scale] if action_scale else []
  action_scale.append(1)

# Get state size
state_size = env.observation_space.n

In [20]:
qtable = np.zeros((state_size, action_size))

In [94]:
def decode_action(action_tuple):
  return sum(tuple(x*y for x, y in zip(action_tuple, action_scale)))

def encode_action(action_scalar):
  num1 = action_scalar//action_scale[0]
  num2 = (action_scalar - num1*action_scale[0])//action_scale[1]
  num3 = action_scalar - num1*action_scale[0] - num2*action_scale[1]
  return (num1, num2, num3)

In [95]:
# Test decode_action and encode_action:
action1 = (0, 0, 0)
action2 = (1,1,1)
action3 = (1, 1, 4)

assert(decode_action(action1) == 0)
assert(decode_action(action2) == 10 + 5 + 1)
assert(decode_action(action3) == 10 + 5 + 4)

assert(encode_action(0) == action1)
assert(encode_action(16) == action2)
assert(encode_action(19) == action3)

In [73]:
type(env.action_space.sample())

tuple

In [92]:
qtable

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [93]:
total_episodes = 15000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005             # Exponential decay rate for exploration prob

In [98]:
rewards = []

# 2 For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = encode_action(np.argmax(qtable[state,:]))

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        action_scalar = decode_action(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action_scalar] = qtable[state, action_scalar] +\
          learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action_scalar])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

Score over time: 0.5938
[[ 4.48644112e+00  2.85822470e+00  5.73360197e+00  2.58433858e-01
  -5.69937576e-02 -3.06816970e-01  3.25837307e+00  2.60613213e-01
   1.95905120e-01  1.59945677e-01 -1.42933770e-01  2.98318009e-01
   5.11389144e-04 -6.13221291e-02 -1.83340203e-01  2.10646081e-01
   2.83302596e+00  1.34180924e-01  1.29262970e-01 -1.03448809e-01]
 [-4.55983127e-01  1.18070034e+00  3.12742515e-01 -7.37025277e-01
   3.89100928e+00  2.54828838e-01  6.83800635e+00 -2.76869129e-01
  -2.66157410e-01  3.94336560e-02 -4.47049518e-02 -1.83567408e-01
  -1.45756091e-01 -7.61042525e-02 -9.46822478e-02 -5.17270063e-02
   2.22457684e-01 -9.84306391e-02  3.36056869e-01  2.46087641e-01]
 [-2.16641992e-01  3.10664723e+00 -1.81055726e-01  3.30976085e-02
   3.13678718e-01  2.36780302e-01  2.86317804e-01  2.89952479e-01
  -1.00186971e-02  4.88609946e+00 -2.15191408e-02  2.98803817e-01
   4.01305797e-01  4.59144342e+00  1.45072884e-01  3.00726391e+00
   9.06918278e-03  2.84650323e-01 -4.10305146e-02 

In [99]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = encode_action(np.argmax(qtable[state,:]))
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
    print("****************************************************\n")
env.close()

****************************************************
EPISODE  0
Total length of input instance: 5, step: 1
Observation Tape    :   CDCDD  
Output Tape         :   E
Targets             :   CDCDD  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: left,
                              write to the output tape: True,
                              prediction: E)
Number of steps 0
****************************************************

****************************************************
EPISODE  1
Total length of input instance: 5, step: 1
Observation Tape    :   DEDCD  
Output Tape         :   C
Targets             :   DEDCD  

Current reward      :   -0.500
Cumulative reward   :   -0.500
Action              :   Tuple(move over input: left,
                              write to the output tape: True,
                              prediction: C)
Number of steps 0
****************************************************

****************

In [120]:
env.reset()
env.render()

Total length of input instance: 3, step: 0
Observation Tape    :   ABB  
Output Tape         :   
Targets             :   ABB  








In [129]:
state, reward, done, info = env.step((0, 1, 0))
print(state, reward, done)
env.render()

1 1.0 False
Total length of input instance: 3, step: 9
Observation Tape    :   ABB  
Output Tape         :   A
Targets             :   ABB  

Current reward      :   1.000
Cumulative reward   :   1.000
Action              :   Tuple(move over input: left,
                              write to the output tape: True,
                              prediction: A)
